## Keras Callbacks

This is to show implementations of Keras Callbacks and how to use them

- Basic History and plotting  
- TensorBoard implementation  
- Model checkpoints  
- Early Stopping  
- Learning Rate Scheduler  
- ReduceLROnPlateau  
- LambdaCallback
- Custom Callback  

### Set up a model

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, Callback, ReduceLROnPlateau, LearningRateScheduler, EarlyStopping, TensorBoard
from keras.callbacks import LambdaCallback
from keras import backend as K

import tensorflow as tf



Using TensorFlow backend.


In [2]:
print(keras.__version__)
print(tf.__version__)

2.0.4
1.1.0


In [3]:
batch_size = 128
num_classes = 10
epochs = 10

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

60000 train samples
10000 test samples


In [4]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [5]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,),name='Dense1'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu',name='Dense2'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Dense1 (Dense)               (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
Dense2 (Dense)               (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

### Basic History and ploting

In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=2,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
def plot_train(hist):
    h = hist.history
    if 'acc' in h:
        meas='acc'
        loc='lower right'
    else:
        meas='loss'
        loc='upper right'
    plt.plot(hist.history[meas])
    plt.plot(hist.history['val_'+meas])
    plt.title('model '+meas)
    plt.ylabel(meas)
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc=loc)

In [ ]:
plot_train(history)

### Model Checkpoints

args 
- filepath - str  can use formating to put epoch number etc {epoc}
- monitor - qunatity to monitor  
- verbose - 0 or 1  
- save_best_only - only save if better than before  
- mode - use 'auto'  
- save_weights_only - if false then will save whole model
- period - the interval between epochs  

In [ ]:
checkpoint = ModelCheckpoint('./checkpoints/weights_{epoch:02d}_{val_acc:.2f}.hdf5', verbose=1, save_best_only=True, mode='auto')

In [ ]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=0,
                    validation_data=(x_test, y_test),
                    callbacks=[checkpoint])

### TensorBoard Callback

args
- log_dir : string , must exist
- histogram_freq : freq in epochs
- write_graph : true or false for seeing the graph in TB  
- write_grads :writing the gradients  not working 2.04
- batch_size : size of batches for histogram  
- write_images :whether to write model weights to see  
- embeddings_freq : 
- embeddings_layer_names :
- embeddings_metadata :

In [ ]:
TB = TensorBoard(log_dir='./logs', histogram_freq=1,
                            write_graph=True,  write_images=False)
#tf.reset_default_graph()

In [ ]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=0,
                    validation_data=(x_test, y_test),
                    callbacks=[TB])

In [ ]:
!tensorboard --logdir=logs

### Early Stopping - stop training when a monitored quanity has stopped improving
args
- monitor : what to monitor 'val_loss', 'acc'
- min_delta :minimum change to qualify as improvement
- patience : number of epocs with no improvement before you stop
- verbose : verbosity mode
- mode : 'auto' , can be 'min' or 'max' determines direction of improvement. Auto = based on monitor

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss',min_delta=0,patience=0,verbose=0,mode='auto')

In [ ]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=[early_stopping])

above stopped after 3 epochs instead of 10 as validation loss didn't get better

### Learning Rate Scheduler
arg
- schedule - this will be function that takes epoch number(int) and returns new Learning Rate (float) 

#### LearningRate = InitialLearningRate * DropRate^floor(Epoch / EpochDrop)

In [12]:
import math

# learning rate schedule for dropping every 10 epochs
def LRDropping(epoch):
    initial_lrate = 0.001
    drop = 0.9
    epochs_drop = 3.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

In [ ]:
LRDropping(2)

In [13]:
LRDrop = LearningRateScheduler(LRDropping)

In [ ]:
K.set_value(model.optimizer.lr,0.001)

In [ ]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=[LRDrop])

In [ ]:
#print out the learning rate
print(K.eval(model.optimizer.lr))

### ReduceLROnPlateau
args
- monitor : quality to be monitored eg. 'val_loss' , 'val_acc'
- factor : the factor by which the current LR be multiplied  
- patience : number of epochs with no improvement  
- verbose : 1 = update messages 0 nothing
- mode : 'auto'  eg. is improvment up or down 'min' 'max'
- epsilon : threshold for measuring the new optimum, to only focus on significant changes  
- cooldown :number of epochs to wait before any new changes
- min_lr: the lowest lr allowed 

In [ ]:
reduce_LR = ReduceLROnPlateau(monitor='val_loss',factor = 0.9, patience=3,cooldown=2, min_lr = 0.00001)

In [ ]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=[reduce_LR])

In [ ]:
#print out the learning rate
print(K.eval(model.optimizer.lr))

### LambdaCallback a way to call anon functions in the callback
args
- on_epoch_being - called at begin of epoch -takes epoch,logs
- on_epoch_end - called at end of epoch -takes epoch,logs
- on_batch_begin _ called a begin of a batch -takes batch,logs
- on_batch_end :-takes epoch,logs
- on_train_begin - -takes logs
- on_train_end - -takes logs

In [21]:
lr_print = LambdaCallback(on_epoch_begin=lambda epoch,logs: print("lr:",K.eval(model.optimizer.lr)))

In [22]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=[lr_print,LRDrop])

Train on 60000 samples, validate on 10000 samples
lr: 0.000729
Epoch 1/10
60000/60000 [==============================] - 0s - loss: 0.0100 - acc: 0.9979 - val_loss: 0.1215 - val_acc: 0.9855
lr: 0.001
Epoch 2/10
60000/60000 [==============================] - 0s - loss: 0.0126 - acc: 0.9973 - val_loss: 0.1440 - val_acc: 0.9832
lr: 0.001
Epoch 3/10
60000/60000 [==============================] - 0s - loss: 0.0074 - acc: 0.9983 - val_loss: 0.1456 - val_acc: 0.9829
lr: 0.0009
Epoch 4/10
60000/60000 [==============================] - 0s - loss: 0.0083 - acc: 0.9981 - val_loss: 0.1342 - val_acc: 0.9858
lr: 0.0009
Epoch 5/10
60000/60000 [==============================] - 0s - loss: 0.0093 - acc: 0.9980 - val_loss: 0.1399 - val_acc: 0.9847
lr: 0.0009
Epoch 6/10
60000/60000 [==============================] - 0s - loss: 0.0068 - acc: 0.9985 - val_loss: 0.1404 - val_acc: 0.9839
lr: 0.00081
Epoch 7/10
60000/60000 [==============================] - 0s - loss: 0.0086 - acc: 0.9981 - val_loss: 0.1417 -